In [1]:
import sys
sys.path.append('../src')

In [2]:
from covid import daily_snapshot
from pathlib import Path

import population as pop
import pandas as pd
import requests

In [3]:
pd.set_option('max_rows', 100) 

In [4]:
date = input('date in mm-dd-yyyy format')

date in mm-dd-yyyy format 04-02-2020


In [5]:
state = 'New Jersey'

### Quick glance at the data

In [6]:
df = daily_snapshot(date)
df.head()

,Unnamed: 0,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key
FIPS,,,,,,,,,,,,
45001.0,NaN,Abbeville,South Carolina,US,2020-04-02 23:25:27,34.223334,-82.461707,6,0,0,0,"Abbeville, South Carolina, US"
22001.0,NaN,Acadia,Louisiana,US,2020-04-02 23:25:27,30.295065,-92.414197,61,1,0,0,"Acadia, Louisiana, US"
51001.0,NaN,Accomack,Virginia,US,2020-04-02 23:25:27,37.767072,-75.632346,10,0,0,0,"Accomack, Virginia, US"
16001.0,NaN,Ada,Idaho,US,2020-04-02 23:25:27,43.452658,-116.241552,312,3,0,0,"Ada, Idaho, US"
19001.0,NaN,Adair,Iowa,US,2020-04-02 23:25:27,41.330756,-94.471059,1,0,0,0,"Adair, Iowa, US"


In [7]:
df[df.Country_Region=='US'].describe()

,Unnamed: 0,Lat,Long_,Confirmed,Deaths,Recovered,Active
count,0.0,2263.000000,2263.000000,2312.000000,2312.000000,2312.000000,2312.0
mean,NaN,37.901839,-90.296335,105.299740,2.563149,3.893166,0.0
std,NaN,5.056066,14.162025,1181.910573,33.492062,187.196122,0.0
min,NaN,13.444300,-159.596679,0.000000,0.000000,0.000000,0.0
25%,NaN,34.226043,-95.801391,2.000000,0.000000,0.000000,0.0
50%,NaN,38.006710,-88.110438,6.000000,0.000000,0.000000,0.0
75%,NaN,41.459743,-82.330657,24.250000,1.000000,0.000000,0.0
max,NaN,64.807262,145.673900,51809.000000,1397.000000,9001.000000,0.0


### Aggregate by state

In [8]:
state_totals = df[df.Country_Region=='US'].groupby(by='Province_State').sum()

In [9]:
state_totals.describe()

,Unnamed: 0,Lat,Long_,Confirmed,Deaths,Recovered,Active
count,58.0,58.000000,58.000000,58.000000,58.000000,58.000000,58.0
mean,0.0,1478.825184,-3523.113909,4197.465517,102.172414,155.189655,0.0
std,0.0,1218.832299,3044.446499,12540.143413,321.174497,1181.889202,0.0
min,0.0,0.000000,-14256.489929,0.000000,0.000000,0.000000,0.0
25%,0.0,398.189159,-5458.201885,316.000000,6.000000,0.000000,0.0
50%,0.0,1233.777309,-3210.628444,853.500000,21.000000,0.000000,0.0
75%,0.0,2469.858479,-881.128482,3003.750000,77.750000,0.000000,0.0
max,0.0,4665.624024,145.673900,92506.000000,2373.000000,9001.000000,0.0


In [10]:
state = 'New Jersey'
state_covid = df[df.Province_State == state]
state_columns=['Admin2', 'Confirmed', 'Deaths', 'Recovered', 'Active', ]

In [11]:
state_covid[state_columns].sort_values(by='Confirmed', ascending=False)

,Admin2,Confirmed,Deaths,Recovered,Active
FIPS,,,,,
90034.0,Unassigned,4866,4,0,0
34003.0,Bergen,4099,120,0,0
34013.0,Essex,2617,99,0,0
34017.0,Hudson,2270,44,0,0
34039.0,Union,2010,34,0,0
34023.0,Middlesex,1766,48,0,0
34031.0,Passaic,1750,22,0,0
34025.0,Monmouth,1458,38,0,0
34029.0,Ocean,1371,38,0,0


In [12]:
state_covid[state_columns].sort_values(by='Deaths', ascending=False)

,Admin2,Confirmed,Deaths,Recovered,Active
FIPS,,,,,
34003.0,Bergen,4099,120,0,0
34013.0,Essex,2617,99,0,0
34023.0,Middlesex,1766,48,0,0
34017.0,Hudson,2270,44,0,0
34027.0,Morris,1082,40,0,0
34025.0,Monmouth,1458,38,0,0
34029.0,Ocean,1371,38,0,0
34039.0,Union,2010,34,0,0
34031.0,Passaic,1750,22,0,0


In [13]:
nj_county_population_url = 'https://www.newjersey-demographics.com/counties_by_population'

In [14]:
nj_pop = pop.nj_county_population(nj_county_population_url)

In [15]:
nj_pop

,county,pop2019
Rank,,
1,Bergen,929999
2,Middlesex,826698
3,Essex,793555
4,Hudson,668631
5,Monmouth,623387
6,Ocean,591939
7,Union,553066
8,Camden,507367
9,Passaic,504041


In [16]:
state_with_population = pd.merge(state_covid[state_columns], nj_pop,left_on='Admin2', right_on='county')

state_with_population['fraction_confirmed'] = state_with_population['Confirmed'] / state_with_population['pop2019']   * 100.0
state_with_population['deaths']             = state_with_population['Deaths']    / state_with_population['pop2019']   * 100.0
state_with_population['death_rate?']        = state_with_population['Deaths']    / state_with_population['Confirmed'] * 100.0

In [17]:
sort_by = input("column to sort Confirmed, Deaths, county, pop2019, fraction_confirmed, deaths, death_rate?")

column to sort Confirmed, Deaths, county, pop2019, fraction_confirmed, deaths, death_rate? fraction_confirmed


In [18]:
state_with_population.sort_values(by=sort_by, ascending=False)

,Admin2,Confirmed,Deaths,Recovered,Active,county,pop2019,fraction_confirmed,deaths,death_rate?
1,Bergen,4099,120,0,0,Bergen,929999,0.440753,0.012903,2.927543
19,Union,2010,34,0,0,Union,553066,0.363429,0.006148,1.691542
15,Passaic,1750,22,0,0,Passaic,504041,0.347194,0.004365,1.257143
8,Hudson,2270,44,0,0,Hudson,668631,0.339500,0.006581,1.938326
6,Essex,2617,99,0,0,Essex,793555,0.329782,0.012476,3.782958
12,Monmouth,1458,38,0,0,Monmouth,623387,0.233884,0.006096,2.606310
14,Ocean,1371,38,0,0,Ocean,591939,0.231612,0.006420,2.771699
13,Morris,1082,40,0,0,Morris,494383,0.218859,0.008091,3.696858
11,Middlesex,1766,48,0,0,Middlesex,826698,0.213621,0.005806,2.718007
17,Somerset,549,17,0,0,Somerset,330176,0.166275,0.005149,3.096539
